In [10]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [11]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [12]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [13]:
from imblearn.combine import SMOTETomek
import numpy as np

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)


(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [14]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [15]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [2]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [11]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/08/11 23:39:00 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/11 23:39:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/667766105493889927/runs/d01f0bd7eabb4a6bb150ba1c36aa23a9.
2024/08/11 23:39:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/667766105493889927.
2024/08/11 23:39:24 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/11 23:39:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://localhost:5000/#/experiments/667766105493889927/runs/f0ba5bc5ff4644c78783c66f2f930ab4.
2024/08/11 23:39:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/6677

### Register the Model

In [1]:
# model_name = 'XGB-Smote'
# run_id=input('Please type RunID')
# model_uri = f"runs:/{run_id}/{model_name}"

# result= mlflow.register_model(model_uri=model_uri, name=model_name)

In [3]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'XGB-Smote'.
Created version '1' of model 'XGB-Smote'.


### Load the Model

In [16]:
# model_path = "mlruns/667766105493889927/55251132916d4b0dab75118ee0d33b72/artifacts/model/"
# loaded_model = mlflow.xgboost.load_model(model_path)
# y_pred = loaded_model.predict(X_test)
# y_pred[:4]


array([0, 0, 0, 0])

In [24]:
import mlflow

for model in mlflow.search_registered_models():
    print(f"Model Name: {model.name}, Versions: {[v.version for v in model.latest_versions]}")


Model Name: XGB-Smote, Versions: [1]
Model Name: test, Versions: []


In [33]:
import mlflow

# Ensure this matches the registered model name in MLflow
model_name = "XGB-Smote"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

# Debug print to ensure the correct model path is being used
print(f"Loading model from URI: {model_uri}")

try:
    # Attempt to load the model using the URI
    loaded_model = mlflow.xgboost.load_model(model_uri)
except Exception as e:
    print(f"Failed to load model from URI: {e}")
    print("Attempting to load model from direct path...")
    
    # Fallback to loading directly from the path if URI fails
    model_path = "mlruns/667766105493889927/55251132916d4b0dab75118ee0d33b72/artifacts/model/"
    print(f"Loading model from path: {model_path}")
    loaded_model = mlflow.xgboost.load_model(model_path)

# Make predictions and print the first four results
y_pred = loaded_model.predict(X_test)
print("Predictions:", y_pred[:4])


Loading model from URI: models:/XGB-Smote/1
Failed to load model from URI: No such file or directory: 'E:\Projects\mlflow_dagshub_demo-main\mlflow_dagshub_demo-main\mlruns\667766105493889927\55251132916d4b0dab75118ee0d33b72\artifacts\model_name\.'
Attempting to load model from direct path...
Loading model from path: mlruns/667766105493889927/55251132916d4b0dab75118ee0d33b72/artifacts/model/
Predictions: [0 0 0 0]


### Transition the Model to Production

In [34]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1723429097478, current_stage='None', description=None, last_updated_timestamp=1723429097478, name='anomaly-detection-prod', run_id='55251132916d4b0dab75118ee0d33b72', run_link=None, source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [38]:
client = mlflow.MlflowClient()
registered_models = client.search_registered_models()
for model in registered_models:
    print(f"Model Name: {model.name}")
    for version in model.latest_versions:
        print(f"  Version: {version.version}, Stage: {version.current_stage}")


Model Name: XGB-Smote
  Version: 1, Stage: None
Model Name: anomaly-detection-prod
  Version: 1, Stage: None
Model Name: test


In [39]:
prod_model_uri = f"models:/{production_model_name}@champion"
print(f"Model URI: {prod_model_uri}")


Model URI: models:/anomaly-detection-prod@champion


In [41]:
import mlflow

# Production model name and version
production_model_name = "anomaly-detection-prod"
model_version = 1

# Attempt to load the model using the "champion" stage
prod_model_uri = f"models:/{production_model_name}@champion"
print(f"Loading model from URI: {prod_model_uri}")

try:
    loaded_model = mlflow.xgboost.load_model(prod_model_uri)
except Exception as e:
    print(f"Failed to load model from URI: {e}")
    print("Attempting to load model from direct path...")

    # Fallback to loading directly from the path
    model_path = "mlruns/667766105493889927/55251132916d4b0dab75118ee0d33b72/artifacts/model/"
    print(f"Loading model from path: {model_path}")
    loaded_model = mlflow.xgboost.load_model(model_path)

# Make predictions and print the first four results
y_pred = loaded_model.predict(X_test)
print("Predictions:", y_pred[:4])


Loading model from URI: models:/anomaly-detection-prod@champion
Failed to load model from URI: No such file or directory: 'E:\Projects\mlflow_dagshub_demo-main\mlflow_dagshub_demo-main\mlruns\667766105493889927\55251132916d4b0dab75118ee0d33b72\artifacts\model_name\.'
Attempting to load model from direct path...
Loading model from path: mlruns/667766105493889927/55251132916d4b0dab75118ee0d33b72/artifacts/model/
Predictions: [0 0 0 0]


In [37]:
model_version = 1
# model_path = "mlruns/667766105493889927/55251132916d4b0dab75118ee0d33b72/artifacts/model/"
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

OSError: No such file or directory: 'E:\Projects\mlflow_dagshub_demo-main\mlflow_dagshub_demo-main\mlruns\667766105493889927\55251132916d4b0dab75118ee0d33b72\artifacts\model_name\.'

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 